![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **Getting Started Code For [Pneumonia Classification in CXRs](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/overview/about) on DPhi**

## Download the images
We are given google drive link in the '[Data](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/data)' section of problem page which has all the required train images (to build the model) and test images to predict the label of these images and submit the predictions on the [DPhi platform](https://dphi.tech/challenges/pneumonia-classification-in-cxrs/76/submit).

We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY/view?usp=sharing

The file id in the above link is: **1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY**

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1d_93d9oFNRBK9Vg6BRxs9wvRbKtNTylY',
                                    dest_path='content/pneumonia_dataset.zip',
                                    unzip=True)

Unzipping...Done.


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [ ]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import accuracy_score
from keras.layers.normalization import BatchNormalization

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'.

In [ ]:
data=[]
img_size = 100
def create_data():
        for item in ['normal','pneumonia']:
            path='/content/content/pneumonia_dataset/train/' + item+"/"
            
            for img in os.listdir(path):         # os.listdir gets you all the list of name of files located in the given path
                try:
                    img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)    # converts the image to pixels and gray scales the images
                    new_img_array=cv2.resize(img_array,(img_size,img_size))
                    # print(img_array)
                    if item == 'normal':
                        data.append([new_img_array,0])
                    else:
                        data.append([new_img_array, 1]) # appending the list of image pixels and respective target value in data
                except Exception as e:
                    pass                                      # try and except is exception handling case in python, saves you from getting errors
                
            
create_data()

In [ ]:
len(data)

2425

In [ ]:
# image pixels of a image
data[2]

[array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,  68,  25,  19],
        [  0,   0,   0, ...,  52,  34,  29],
        ...,
        [  0,   0,   0, ..., 114,  65,  17],
        [  0,   0,   0, ..., 134,  66,  19],
        [  0,   0,   0, ..., 133,  69,  20]], dtype=uint8), 0]

#### Shuffle the data

In [ ]:
np.random.shuffle(data)

#### Separating the images and labels


In [ ]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1]), array([1280, 1145]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [ ]:
x =  x.reshape(-1, 100, 100, 1)

In [ ]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [ ]:
X_train.shape

(1697, 100, 100, 1)

In [ ]:
X_train =  X_train.reshape(-1, 100, 100, 1)

In [ ]:
X_train.shape

(1697, 100, 100, 1)

In [ ]:
X_train[0]

array([[[173],
        [ 52],
        [  7],
        ...,
        [ 11],
        [ 12],
        [ 13]],

       [[ 80],
        [  7],
        [  0],
        ...,
        [  3],
        [  3],
        [  3]],

       [[ 79],
        [  5],
        [  0],
        ...,
        [  3],
        [  3],
        [  3]],

       ...,

       [[ 29],
        [  0],
        [  0],
        ...,
        [  0],
        [  0],
        [  0]],

       [[ 32],
        [  1],
        [  0],
        ...,
        [  0],
        [  0],
        [  0]],

       [[ 42],
        [  6],
        [  0],
        ...,
        [  0],
        [  0],
        [  1]]], dtype=uint8)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [ ]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
54/54 [==============================] - 1s 11ms/step - loss: 30.1966 - accuracy: 0.5282
Epoch 2/10
54/54 [==============================] - 1s 10ms/step - loss: 0.6461 - accuracy: 0.6057
Epoch 3/10
54/54 [==============================] - 1s 10ms/step - loss: 0.5403 - accuracy: 0.7305
Epoch 4/10
54/54 [==============================] - 1s 10ms/step - loss: 0.4620 - accuracy: 0.7935
Epoch 5/10
54/54 [==============================] - 1s 10ms/step - loss: 0.3798 - accuracy: 0.8309
Epoch 6/10
54/54 [==============================] - 1s 10ms/step - loss: 0.3230 - accuracy: 0.8632
Epoch 7/10
54/54 [==============================] - 1s 10ms/step - loss: 0.2562 - accuracy: 0.8952
Epoch 8/10
54/54 [==============================] - 1s 10ms/step - loss: 0.2047 - accuracy: 0.9171
Epoch 9/10
54/54 [==============================] - 1s 10ms/step - loss: 0.1562 - accuracy: 0.9509
Epoch 10/10
54/54 [==============================] - 1s 10ms/step - loss: 0.1242 - accuracy: 0.9557


## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [ ]:
cnn.evaluate(X_val, y_val)

23/23 [==============================] - 0s 5ms/step - loss: 1.4499 - accuracy: 0.6168


[1.449948787689209, 0.6167582273483276]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data.

#### Load Test Set
Load the test data on which final submission is to be made.

In [ ]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/content/pneumonia_dataset/test.csv")
test_image_order.head()

,filename
0,CXR_test_519.png
1,CXR_test_578.png
2,CXR_test_359.png
3,CXR_test_573.png
4,CXR_test_471.png


#### Getting images file path

In [ ]:
file_paths = [[fname, '/content/content/pneumonia_dataset/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [ ]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  606 matches the number of file paths i.e.  606


#### Converting the file_paths to dataframe

In [ ]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,CXR_test_519.png,/content/content/pneumonia_dataset/test/CXR_te...
1,CXR_test_578.png,/content/content/pneumonia_dataset/test/CXR_te...
2,CXR_test_359.png,/content/content/pneumonia_dataset/test/CXR_te...
3,CXR_test_573.png,/content/content/pneumonia_dataset/test/CXR_te...
4,CXR_test_471.png,/content/content/pneumonia_dataset/test/CXR_te...


## Data Pre-processing on test_data


In [ ]:
test_pixel_data = []     # initialize an empty numpy array
for i in range(len(test_images)):
  
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale
  new_img_array=cv2.resize(img_array,(img_size,img_size))
  test_pixel_data.append(new_img_array)

In [ ]:
test_pixel_data = np.asarray(test_pixel_data)

In [ ]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

In [ ]:
test_pixel_data

array([[[[  3],
         [  3],
         [  3],
         ...,
         [  5],
         [  6],
         [  7]],

        [[  3],
         [  2],
         [  2],
         ...,
         [  5],
         [  6],
         [  6]],

        [[  4],
         [  2],
         [  2],
         ...,
         [  5],
         [  5],
         [  6]],

        ...,

        [[  7],
         [  5],
         [  3],
         ...,
         [100],
         [ 85],
         [ 64]],

        [[  7],
         [  4],
         [  3],
         ...,
         [106],
         [ 97],
         [ 71]],

        [[  5],
         [  3],
         [  3],
         ...,
         [103],
         [ 94],
         [ 72]]],


       [[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [

### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
pred = cnn.predict(test_pixel_data)

In [ ]:
pred

array([[2.82310754e-01],
       [4.69162920e-03],
       [3.59779596e-01],
       [2.43256986e-01],
       [1.74410138e-02],
       [6.19920969e-01],
       [2.71303624e-01],
       [1.12559181e-03],
       [2.72300631e-01],
       [1.77883923e-01],
       [1.79590628e-01],
       [4.84948725e-01],
       [3.07128161e-01],
       [9.06135201e-01],
       [5.39903976e-02],
       [3.05024654e-01],
       [8.85919780e-02],
       [1.83353394e-01],
       [9.97327089e-01],
       [9.85914469e-01],
       [2.34587327e-01],
       [8.11746299e-01],
       [7.23944083e-02],
       [9.91784096e-01],
       [7.29476154e-01],
       [1.68014511e-01],
       [9.99260604e-01],
       [4.13237751e-01],
       [1.77420005e-01],
       [9.30237651e-01],
       [1.27766598e-02],
       [6.64514601e-01],
       [2.29345206e-02],
       [2.85983481e-03],
       [1.48583623e-03],
       [7.73266315e-01],
       [6.93531215e-01],
       [4.16645050e-01],
       [9.99712527e-01],
       [6.93133712e-01],


Convert the numerical classes to corresponding categorical classes.

In [ ]:
predictions = []
for item in pred:
  if item <= 0.5:
    predictions.append('normal')
  else:
    predictions.append('pneumonia')

In [ ]:
predictions

['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 'pneumonia',
 'normal',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'normal',
 'pneumonia',
 'pneumonia',
 

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/pneumonia-classification-challenge-by-segmind/76/overview/about)** to make the submission.